# Extracting & Plotting  Feature Names & Importance from Scikit-Learn Pipelines

If you have ever been tasked with productionalizing a machine learning model, you probably know that Scikit-Learn library offers one of the best ways -- if not the best way -- of creating production-quality machine learning workflows. The ecosystem's [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html), [ColumnTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), [preprocessors](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing), [imputers](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.impute) & [feature selection](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.feature_selection) classes are powerful tools that transform raw data into model-ready features.

However, before anyone is going to let you deploy to production, you are going to want to have some minimal understanding of how the new model works. The most common way to explain how a black-box model works is by plotting feature names and importance values. If you have ever tried to extract the feature names from a heterogeneous dataset processed by ColumnTransformer, you know that this is no easy task. Exhaustive Internet searches have only brought to my attention where others have [asked](https://github.com/scikit-learn/scikit-learn/issues/6424) [the](https://github.com/scikit-learn/scikit-learn/pull/6431) [same](https://github.com/scikit-learn/scikit-learn/pull/12627) [question](https://github.com/scikit-learn/scikit-learn/pull/13307) or offered a [partial answer](https://github.com/scikit-learn/scikit-learn/issues/12525), instead of yielding a comprehensive and satisfying solution. 

To remedy this situation, I have developed a class called `FeatureImportance` that will extract feature names and importance values from a Pipeline instance. It then uses the Plotly library to plot the feature importance using only a few lines of code. In this post, I will load a fitted Pipeline, demonstrate how to use my class and then give an overview of how it works. The complete code can be found [here](https://www.kaggle.com/kylegilde/feature-importance) or at the end of this blog post.

There are two things I should note before continuing:

1. I credit Joey Gao's code on [this thread](https://github.com/scikit-learn/scikit-learn/issues/12525#issuecomment-436217100) with showing the way to tackle this problem.

2. My post assumes that you have worked with Scikit-Learn and Pandas before and are familiar with how ColumnTransformer, Pipeline & preprocessing classes facilitate reproducible feature engineering processes. If you need a refresher, check out this [Scikit-Learn example](https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer_mixed_types.html).

## Creating a Pipeline


For the purposes of demonstration, I've written a script called [fit_pipeline_ames.py](https://www.kaggle.com/kylegilde/fit-pipeline-ames). It loads the [Ames housing training data from Kaggle](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data) and fits a moderately complex Pipeline. The `pipe` instance contains the following 4 steps:

1. The [ColumnTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) instance is composed of 3 Pipelines, containing a total of 4 transformer instances, including [SimpleImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html), [OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) & [GLMMEncoder](http://contrib.scikit-learn.org/category_encoders/glmm.html) from the [category_encoders](https://contrib.scikit-learn.org/category_encoders/) package. See my [previous blog post](https://towardsdatascience.com/building-columntransformers-dynamically-1-6354bd08aa54) for a full explanation of how I dynamically constructed this particular ColumnTransformer.

2. The [VarianceThreshold](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.VarianceThreshold.html) uses the default threshold of 0, which removes any features that contain only a single value. Some models will fail if a feature has no variance.

3. The [SelectPercentile](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectPercentile.html) uses the [f_regression](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_regression.html) scoring function with a percentile threshold of 90. These settings retain the top 90% of features and discard the bottom 10%.

4. The [CatBoostRegressor](https://catboost.ai/docs/concepts/python-reference_catboostregressor.html) model is fit to the `SalesPrice` dependent variable using the features created and selected in the preceding steps.

In [4]:
from fit_pipeline_ames import *
pipe

Pipeline(memory=None,
         steps=[('column_transformer',
                 ColumnTransformer(n_jobs=4, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('numeric_pipeline',
                                                  Pipeline(memory=None,
                                                           steps=[('simpleimputer',
                                                                   SimpleImputer(add_indicator=True,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                        

## Plotting FeatureImportance


With the help of FeatureImportance, we can extract the feature names and importance values and plot them with 3 lines of code.

In [10]:
%%capture
from feature_importance import FeatureImportance
feature_importance = FeatureImportance(pipe)
feature_importance.plot(top_n_features=25)

The `plot` method takes a number of arguments that control the plot's display. The most important ones are the following:

- `top_n_features`: This controls how many features will be plotted. The default value is 100. The plot's title will indicate this value as well as how many features there are in total. To plot all features, just set `top_n_features` to a number larger than the total features. 

- `rank_features`: This argument controls whether the integer ranks are displayed in front of the feature names. The default is `True`. I find that this aids with interpretation, especially when comparing the feature importance from multiple models.

- `max_scale`: This determines whether the importance values are scaled by the maximum value & multiplied by 100. The default is `True`. I find that this enables an intuitive way to compare how important other features are vis-a-viz the most important one. For instance, in the plot of above, we can say that `GrLivArea` is about 81% as important to the model as the top feature, `OverallQty`.

## How It Works

The `FeatureImportance` class should be instantiated using a fitted Pipeline instance. (You can also change the `verbose` argument to `True` if you want to have all of the diagnostics printed to your console.) My class validates that this Pipeline starts with a `ColumnTransformer` instance and ends with a regression or classification model that has the `feature_importance_` attribute. As intermediate steps, the Pipeline can have any number or no instances of classes from [sklearn.feature_selection](https://scikit-learn.org/stable/modules/feature_selection.html).

The `FeatureImportance` class is composed of 4 methods.

1. `get_feature_names_from_col_transformer` was the hardest method to devise. It iterates through the `ColumnTransformer` transformers, uses the `hasattr` function to discern what type of class we are dealing with and pulls the feature names accordingly. (Special Note: If the ColumnTransformer contains Pipelines and if one of the transformers in the Pipeline is adding completely new columns, it must come last in the pipeline. For example, OneHotEncoder, [MissingIndicator](https://scikit-learn.org/stable/modules/generated/sklearn.impute.MissingIndicator.html) & SimpleImputer(add_indicator=True) add columns to the dataset that didn't exist before, so they should come last in the Pipeline.)

2. `get_selected_features` calls `get_feature_names_from_col_transformer`. Then it tests for whether the main Pipeline contains any classes from sklearn.feature_selection based upon the existence of the `get_support` method. If it does, this method returns the features names that were retained by the selector class or classes. It also saves a list of not-selected features in the attribute `discarded_features`.

3. `get_feature_importance` calls `get_selected_features` and then creates a Pandas Series where values are the feature importance values from the model and its index is the feature names created by the first 2 methods. This Series is then stored in the `feature_importance` attribute.

4. `plot` calls `get_feature_importance` and plots the output based upon the specifications.

## Complete Code

The complete code is shown below and can be found here. If you create a Pipeline that you believe should be supported by FeatureImportance but is not, please provide a reproducible example, and I will consider making the necessary changes. 

The original notebook for this blog post can be found [here](https://www.kaggle.com/kylegilde/extracting-scikit-feature-names-importances). Stay tuned for further posts on training & regularizing models with Scikit-Learn ColumnTransformers and Pipelines. Let me know if you found this post helpful or have any ideas for improvement. Thanks!